In [1]:
import os
import sqlite3
import numpy as np
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langsmith import Client 
from langsmith import traceable
import base64
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import AzureChatOpenAI
from io import BytesIO
from PIL import Image
import pandas as pd
from langchain_core.output_parsers import JsonOutputParser
from typing import List, Dict
import json
import re
import asyncio
from tqdm.asyncio import tqdm
import torch
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever
from processing_scraped_docs_consolidated_script import vectordb_path,collection_name

torch.classes.__path__ = []

from dotenv import load_dotenv

load_dotenv() 

# Environment Variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_TRACING'] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "Confluence Retriever"
# os.environ["LANGCHAIN_API_KEY"] = str(os.environ.get("LANGCHAIN_API_KEY"))
os.environ["LANGSMITH_API_KEY"] = str(os.environ.get("LANGCHAIN_API_KEY"))
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

client = Client()

AZURE_OPENAI_VERSION=os.environ.get("AZURE_OPENAI_VERSION")
AZURE_OPENAI_DEPLOYMENT=os.environ.get("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_ENDPOINT=os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY=os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_DEPLOYMENT_GPT4=os.environ.get("AZURE_OPENAI_DEPLOYMENT_GPT4")

gpt_35 = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    temperature=0.75,
    )

gpt_4o = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    azure_deployment="gpt-4o",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    temperature=0.6,
    )


/home/sourjyamukherjee/Projects/Confluence_Retriever/Confluence_Retriever/processing_scraped_docs_consolidated_script.py:39: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5",
/home/sourjyamukherjee/Projects/Confluence_Retriever/Confluence_Retriever/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sourjyamukherjee/Projects/Confluence_Retriever/Confluence_Retriever/processing_scraped_docs_con

In [2]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5", model_kwargs={"device": "cpu"})
vectordb = Chroma(persist_directory=vectordb_path, collection_name=collection_name, embedding_function=embedding_model)
# all_child_docs = vectordb._collection.get()
all_child_docs = vectordb.get()
bm25_retriever = BM25Retriever.from_documents(all_child_docs)

/tmp/ipykernel_72207/2639565893.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=vectordb_path, collection_name=collection_name, embedding_function=embedding_model)


AttributeError: 'str' object has no attribute 'page_content'

In [20]:
all_child_docs = vectordb._collection.get(include=[ "documents","metadatas"],limit=5)

In [21]:
from langchain_core.documents import Document

def convert_chroma_response_to_docs(chroma_response):
    """
    Converts Chroma's get() method response into a list of LangChain Document objects.
    
    Args:
        chroma_response (dict): The dictionary response from ChromaDB's get() method.
    
    Returns:
        List[Document]: A list of Document objects with `page_content` and `metadata`.
    """
    documents = chroma_response.get("documents", [])
    metadatas = chroma_response.get("metadatas", [])

    # Ensure both lists are of the same length
    if len(documents) != len(metadatas):
        raise ValueError("Mismatch between number of documents and metadata entries.")

    return [Document(page_content=doc, metadata=meta) for doc, meta in zip(documents, metadatas)]


In [ ]:
all_child_docs = 

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [22]:
all_child_docs=convert_chroma_response_to_docs(all_child_docs)

In [23]:
all_child_docs

[Document(metadata={'attachments': "['- ./md_output/Sponsor  Ads/sponsorads-sequence diagram.png', '- ./md_output/Sponsor  Ads/sponsorads-Class diagram.jpg', '- ./md_output/Sponsor  Ads/sponsorads-sequence diagram.jpg', '- ./md_output/Sponsor  Ads/Class diagram.jpg']", 'source': 'https://bigbasket.atlassian.net/wiki/rest/api/content/719290385', 'title': 'Sponsor  Ads'}, page_content='| Bigbasket APP | CRUD for Sponsor ads Registration Existing business logic for banner ads to support sponsor ads in all the flows like ,  Create Update  Bulk Download report, Page to show success and failure report with links to download View View All Validation position upload Date range upload | Create/update - build support of ad create/update for promotion type - add promotion type in ad position and date ranges tables - top l3 categories check, check for keyword is product is belongs to top l3 categories - banner ad check  - SKU id/product id information - Add can set up for parent or child product -

In [18]:
dense_retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={'k': 5})
docs = dense_retriever.invoke("planogram")

In [19]:
docs

[Document(metadata={'attachments': '[]', 'parent_id': 80944137.0, 'source': 'https://bigbasket.atlassian.net/wiki/rest/api/content/719454209', 'title': 'Planogram MoM and Q&A'}, page_content='### Title: Planogram MoM and Q&A\n# Q&A'),
 Document(metadata={'attachments': '[]', 'parent_id': 80944137.0, 'source': 'https://bigbasket.atlassian.net/wiki/rest/api/content/719454209', 'title': 'Planogram MoM and Q&A'}, page_content="### Title: Planogram MoM and Q&A\n9. Two different API's are provided by BBAdmin; one for bulk picking and another for packed sku's. In packed sku's api, planogram need to get the parent sku and return the location for picking.\n10. In CC, picking always happens in crates to the nearest roundoff of the requested weight/quantity in pcs.\n11. In CC, crates are stacked in planogram only via receiving.\n    1. Enhancement: Support a random crate with a barcode label to enter into planogram via TI operation.\n12. In DC, F&V are received in crates and stacked in crates in 

In [3]:
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 1.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 KB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 KB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 KB 931.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 KB 3.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
import spacy
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Load spaCy model - you'll need to install it with: python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

def extract_keywords(query, method="hybrid"):
    """
    Extract meaningful keywords from a query using multiple techniques.
    
    Args:
        query: The user query string
        method: Extraction method - "spacy", "tfidf", or "hybrid" (default)
    
    Returns:
        List of extracted keywords
    """
    # Clean the query
    clean_query = re.sub(r'[^\w\s]', ' ', query.lower())
    
    if method == "spacy":
        # Use spaCy for linguistic-based extraction
        doc = nlp(clean_query)
        
        # Extract nouns, proper nouns, and technical terms
        keywords = []
        for token in doc:
            # Include nouns and proper nouns
            if token.pos_ in ["NOUN", "PROPN"]:
                keywords.append(token.text)
            
            # Include verbs that might be technical actions
            if token.pos_ == "VERB" and len(token.text) > 3:  # Avoid common short verbs
                keywords.append(token.text)
                
        # Extract named entities
        for ent in doc.ents:
            keywords.append(ent.text)
            
        # Extract noun chunks (noun phrases)
        for chunk in doc.noun_chunks:
            keywords.append(chunk.text)
            
    elif method == "tfidf":
        # Use TF-IDF for statistical keyword extraction
        # This works better with a corpus, but we can use a simple version
        vectorizer = TfidfVectorizer(
            max_df=0.9, 
            min_df=1,
            stop_words='english',
            use_idf=True
        )
        
        # Create a small corpus with the query
        corpus = [clean_query]
        
        # Add some general text to help with IDF calculations
        corpus.extend([
            "password credentials login authentication",
            "configuration settings setup installation",
            "database server network connection",
            "user account profile settings"
        ])
        
        # Fit and transform
        tfidf_matrix = vectorizer.fit_transform(corpus)
        
        # Get feature names
        feature_names = vectorizer.get_feature_names_out()
        
        # Get scores for the query (first document)
        scores = tfidf_matrix[0].toarray()[0]
        
        # Get top scoring terms
        top_indices = scores.argsort()[-10:][::-1]  # Get top 10 terms
        keywords = [feature_names[i] for i in top_indices]
        
    else:  # hybrid approach
        # Combine both methods
        spacy_keywords = extract_keywords(query, "spacy")
        tfidf_keywords = extract_keywords(query, "tfidf")
        
        # Combine and count occurrences
        all_keywords = spacy_keywords + tfidf_keywords
        keyword_counts = Counter(all_keywords)
        
        # Get keywords that appear in both methods or have high counts
        keywords = [k for k, c in keyword_counts.items() if c > 1]
        
        # Add any technical terms that might have been missed
        technical_terms = extract_technical_terms(query)
        keywords.extend(technical_terms)
        
        # If we don't have enough keywords, add top terms from either method
        if len(keywords) < 3:
            remaining = set(all_keywords) - set(keywords)
            keywords.extend(list(remaining)[:5])
    
    # Remove duplicates and normalize
    unique_keywords = []
    seen = set()
    for keyword in keywords:
        normalized = keyword.lower().strip()
        if normalized and normalized not in seen and len(normalized) > 2:
            seen.add(normalized)
            unique_keywords.append(normalized)
    
    return unique_keywords

def extract_technical_terms(query):
    """Extract technical terms that might be missed by other methods"""
    technical_patterns = [
        r'password[s]?',
        r'credential[s]?',
        r'token[s]?',
        r'api[_\s]?key[s]?',
        r'secret[s]?',
        r'config(?:uration)?[s]?',
        r'database[s]?|db[s]?',
        r'server[s]?',
        r'endpoint[s]?',
        r'url[s]?',
        r'[a-zA-Z0-9_]+\.(?:py|js|java|cpp|rb|go|rs|php|html|css|json|yaml|yml|xml|md|txt)'  # file extensions
    ]
    
    technical_terms = []
    for pattern in technical_patterns:
        matches = re.finditer(pattern, query, re.IGNORECASE)
        for match in matches:
            technical_terms.append(match.group(0))
    
    return technical_terms

In [16]:
query = "give me the password and credentils for newrelic and grafana dashboards"
query = "give me steps for blocking a url at jdvar gateway"
query = "tell me about the stacking process at bigbasket planogram"
extract_keywords(query,"hybrid")

['tell', 'stacking', 'process', 'bigbasket', 'planogram']

In [ ]:
df = pd.read_csv("child_chunks_400")